## 概要
### 手写数字识别任务
数字识别是计算机从纸质文档、照片或其他来源接收、理解并识别可读的数字的能力，目前比较受关注的是手写数字识别。手写数字识别是一个典型的图像分类问题，已经被广泛应用于汇款单号识别、手写邮政编码识别等领域，大大缩短了业务处理时间，提升了工作效率和质量。

在处理如 **图1** 所示的手写邮政编码的简单图像分类任务时，可以使用基于MNIST数据集的手写数字识别模型。MNIST 是深度学习领域标准、易用的成熟数据集，包含 50000 条训练样本和 10000 条测试样本。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/04ab1f9d699e40659a4b69ee069a5136a15cb04bb9d848c2be536da68a8abe5e" width="800"></center>
<center><br>图1：手写数字识别任务示意图</br></center>

* 任务输入：一系列手写数字图片，其中每张图片都是28x28的像素矩阵。
* 任务输出：经过了大小归一化和居中处理，输出对应的0~9的数字标签。

## MNIST 数据集简介
MNIST 数据集是从 NIST 的 Special Database 3（SD-3）和 Special Database 1（SD-1）构建而来。Yann LeCun 等人从 SD-1 和 SD-3 中各取一半数据作为 MNIST 训练集和测试集，其中训练集来自 250 位不同的标注员，且训练集和测试集的标注员完全不同。

MNIST 数据集的发布，吸引了大量科学家训练模型。1998年，LeCun 分别用单层线性分类器、多层感知器（Multilayer Perceptron, MLP）和多层卷积神经网络 LeNet进行实验，使得测试集的误差不断下降（从12%下降到0.7%）。在研究过程中，LeCun 提出了卷积神经网络（Convolutional Neural Network，CNN），大幅度地提高了手写字符的识别能力，也因此成为了深度学习领域的奠基人之一。

手写数字识别的模型是深度学习中相对简单的模型，非常适用初学者。正如学习编程时，我们输入的第一个程序是打印“Hello World！”一样。 在飞桨的入门教程中，我们选取了手写数字识别模型作为启蒙教材，以便更好的帮助读者快速掌握飞桨平台的使用。

## 导入环境
在正式开始实验之前，让我们看看学习本章内容需要使用到的 python 模块。
* os 模块：主要用于处理文件和目录，比如：获取当前目录下文件，删除指定文件，改变目录，查看文件大小等；
* json 模块：用于解析 json 格式的文件；
* gzip 模块：用于解压以 gz 结尾的文件；
* numpy 模块：是常用的科学计算库；
* random 模块：是 python 自身的随机化模块，这里主要用于对数据集进行乱序操作；
* time 模块：主要用于处理时间系列的数据，在该实验主要用于返回当前时间戳，计算脚本每个epoch运行所需要的时间；
* paddle 模块：是百度推出的深度学习开源框架；

In [ ]:
import os
import json
import gzip
import numpy as np
import random
import time
import paddle

## 数据准备
在工业实践中，我们面临的任务和数据环境千差万别，通常需要自己编写适合当前任务的数据处理程序，一般涉及如下五个环节：
* 读入数据
* 数据划分
* 数据乱序
* 数据校验
* 异步读取

### 读入数据并划分数据集
实际应用中，保存到本地的数据存储格式多种多样，如MNIST数据集以json格式存储在本地，其数据存储结构如 **图3** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/7075f5ca75c54e4e8553c10b696913a1a178dad37c5c460a899cd75635cd7961" width="500" hegiht="" ></center>
<center><br>图3：MNIST数据集的存储结构</br></center>
<br></br>

**data**包含三个元素的列表：train_set、val_set、 test_set，包括50000条训练样本、10000条验证样本、10000条测试样本。每个样本包含手写数字图片和对应的标签。

* **train_set（训练集）**：用于确定模型参数。
* **val_set（验证集）**：用于调节模型超参数（如多个网络结构、正则化权重的最优选择）。
* **test_set（测试集）**：用于估计应用效果（没有在模型中应用过的数据，更贴近模型在真实场景应用的效果）。

**train_set**包含两个元素的列表：train_images、train_labels。

* **train_images**：[50000, 784]的二维列表，包含50000张图片。每张图片用一个长度为784的向量表示，内容是 $28 \times 28$ 尺寸的像素灰度值（黑白图片）。
* **train_labels**：[50000, ]的列表，表示这些图片对应的分类标签，即0~9之间的一个数字。

在本地`./work/datasets`目录下读取文件名称为`mnist.json.gz`的MNIST数据，并拆分成训练集、验证集和测试集，实现方法如下所示。

In [ ]:
# 声明数据集文件位置
datafile = './work/datasets/mnist.json.gz'
print('loading mnist dataset from {} ......'.format(datafile))
# 加载json数据文件
data = json.load(gzip.open(datafile))
print('mnist dataset load done')
# 读取到的数据区分训练集，验证集，测试集
train_set, val_set, eval_set = data

# 观察训练集数据
imgs, labels = train_set[0], train_set[1]
print("训练数据集数量: ", len(imgs))

# 观察验证集数量
imgs, labels = val_set[0], val_set[1]
print("验证数据集数量: ", len(imgs))

# 观察测试集数量
imgs, labels = val= eval_set[0], eval_set[1]
print("测试数据集数量: ", len(imgs))

loading mnist dataset from ./work/datasets/mnist.json.gz ......
mnist dataset load done
训练数据集数量:  50000
验证数据集数量:  10000
测试数据集数量:  10000


### 数据乱序
* **训练样本乱序：** 先将样本按顺序进行编号，建立ID集合index_list。然后将index_list乱序，最后按乱序后的顺序读取数据。

------
**说明：**

通过大量实验发现，模型对最后出现的数据印象更加深刻。训练数据导入后，越接近模型训练结束，最后几个批次数据对模型参数的影响越大。为了避免模型记忆影响训练效果，需要进行样本乱序操作。

------
* **生成批次数据：** 先设置合理的 batch size，再将数据转变成符合模型输入要求的 np.array 格式返回。同时，在返回数据时将 Python 生成器设置为``yield``模式，以减少内存占用。

在执行如上两个操作之前，需要先将数据处理代码封装成load_data函数，方便后续调用。load_data有三种模式：``train``、``valid``、``eval``，分为对应返回的数据是训练集、验证集、测试集。

In [ ]:
imgs, labels = train_set[0], train_set[1]
print("训练数据集数量: ", len(imgs))
# 获得数据集长度
imgs_length = len(imgs)
# 定义数据集每个数据的序号，根据序号读取数据
index_list = list(range(imgs_length))
# 读入数据时用到的批次大小
BATCHSIZE = 100

# 随机打乱训练数据的索引序号
random.shuffle(index_list)

# 定义数据生成器，返回批次数据
def data_generator():
    imgs_list = []
    labels_list = []
    for i in index_list:
        # 将数据处理成希望的类型
        img = np.array(imgs[i]).astype('float32')
        label = np.array(labels[i]).astype('float32')
        imgs_list.append(img) 
        labels_list.append(label)
        if len(imgs_list) == BATCHSIZE:
            # 获得一个batchsize的数据，并返回
            yield np.array(imgs_list), np.array(labels_list)
            # 清空数据读取列表
            imgs_list = []
            labels_list = []

    # 如果剩余数据的数目小于BATCHSIZE，
    # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
    if len(imgs_list) > 0:
        yield np.array(imgs_list), np.array(labels_list)
    return data_generator

训练数据集数量:  50000


In [ ]:
# 声明数据读取函数，从训练集中读取数据
train_loader = data_generator
# 以迭代的形式读取数据
for batch_id, data in enumerate(train_loader()):
    image_data, label_data = data
    if batch_id == 0:
        # 打印数据shape和类型
        print("打印第一个batch数据的维度:")
        print("图像维度: {}, 标签维度: {}".format(image_data.shape, label_data.shape))
    break

打印第一个batch数据的维度:
图像维度: (100, 784), 标签维度: (100,)


### 数据校验
在实际应用中，原始数据可能存在标注不准确、数据杂乱或格式不统一等情况。因此在完成数据处理流程后，还需要进行数据校验，一般有两种方式：

* 机器校验：加入一些校验和清理数据的操作。
* 人工校验：先打印数据输出结果，观察是否是设置的格式。再从训练的结果验证数据处理和读取的有效性。

#### 机器校验

如下代码所示，如果数据集中的图片数量和标签数量不等，说明数据逻辑存在问题，可使用assert语句校验图像数量和标签数据是否一致。

In [ ]:
imgs_length = len(imgs)
assert len(imgs) == len(labels), "length of train_imgs({}) should be the same as train_labels({})".format(len(imgs), len(label))

#### 人工校验

人工校验是指打印数据输出结果，观察是否是预期的格式。实现数据处理和加载函数后，我们可以调用它读取一次数据，观察数据的shape和类型是否与函数中设置的一致。

In [ ]:
# 声明数据读取函数，从训练集中读取数据
train_loader = data_generator
# 以迭代的形式读取数据
for batch_id, data in enumerate(train_loader()):
    image_data, label_data = data
    if batch_id == 0:
        # 打印数据shape和类型
        print("打印第一个batch数据的维度，以及数据的类型:")
        print("图像维度: {}, 标签维度: {}, 图像数据类型: {}, 标签数据类型: {}".format(image_data.shape, label_data.shape, type(image_data), type(label_data)))
    break

打印第一个batch数据的维度，以及数据的类型:
图像维度: (100, 784), 标签维度: (100,), 图像数据类型: <class 'numpy.ndarray'>, 标签数据类型: <class 'numpy.ndarray'>


### 封装数据准备函数
上文，我们从读取数据、划分数据集、到打乱训练数据、构建数据读取器以及数据校验，完成了一整套一般性的数据处理流程，下面将这些步骤放在一个函数中实现，方便在神经网络训练时直接调用。

In [ ]:
def load_data(mode='train'):
    datafile = './work/mnist.json.gz'
    print('loading mnist dataset from {} ......'.format(datafile))
    # 加载json数据文件
    data = json.load(gzip.open(datafile))
    print('mnist dataset load done')
   
    # 读取到的数据区分训练集，验证集，测试集
    train_set, val_set, eval_set = data
    if mode=='train':
        # 获得训练数据集
        imgs, labels = train_set[0], train_set[1]
    elif mode=='valid':
        # 获得验证数据集
        imgs, labels = val_set[0], val_set[1]
    elif mode=='eval':
        # 获得测试数据集
        imgs, labels = eval_set[0], eval_set[1]
    else:
        raise Exception("mode can only be one of ['train', 'valid', 'eval']")
    print("训练数据集数量: ", len(imgs))
    
    # 校验数据
    imgs_length = len(imgs)

    assert len(imgs) == len(labels), \
          "length of train_imgs({}) should be the same as train_labels({})".format(len(imgs), len(labels))
    
    # 获得数据集长度
    imgs_length = len(imgs)

    # 定义数据集每个数据的序号，根据序号读取数据
    index_list = list(range(imgs_length))
    # 读入数据时用到的批次大小
    BATCHSIZE = 100
    
    # 定义数据生成器
    def data_generator():
        if mode == 'train':
            # 训练模式下打乱数据
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        for i in index_list:
            # 将数据处理成希望的类型
            img = np.array(imgs[i]).astype('float32')
            label = np.array(labels[i]).astype('float32')
            imgs_list.append(img) 
            labels_list.append(label)
            if len(imgs_list) == BATCHSIZE:
                # 获得一个batchsize的数据，并返回
                yield np.array(imgs_list), np.array(labels_list)
                # 清空数据读取列表
                imgs_list = []
                labels_list = []
    
        # 如果剩余数据的数目小于BATCHSIZE，
        # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
        if len(imgs_list) > 0:
            yield np.array(imgs_list), np.array(labels_list)
    return data_generator

### 异步读取
上面提到的数据读取采用的是同步数据读取方式。对于样本量较大、数据读取较慢的场景，建议采用异步数据读取方式。异步读取数据时，数据读取和模型训练并行执行，从而加快了数据读取速度，牺牲一小部分内存换取数据读取效率的提升，二者关系如 **图4** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/a5fd990c5355426183a71b95aa28a59f979014f6905144ddb415c5a4fe647441" width="500" ></center>
<center><br>图4：同步数据读取和异步数据读取示意图</br></center>
<br></br>

* **同步数据读取**：数据读取与模型训练串行。当模型需要数据时，才运行数据读取函数获得当前批次的数据。在读取数据期间，模型一直等待数据读取结束才进行训练，数据读取速度相对较慢。
* **异步数据读取**：数据读取和模型训练并行。读取到的数据不断的放入缓存区，无需等待模型训练就可以启动下一轮数据读取。当模型训练完一个批次后，不用等待数据读取过程，直接从缓存区获得下一批次数据进行训练，从而加快了数据读取速度。
* **异步队列**：数据读取和模型训练交互的仓库，二者均可以从仓库中读取数据，它的存在使得两者的工作节奏可以解耦。

使用飞桨实现异步数据读取只需要两个步骤：
1. 构建一个继承paddle.io.Dataset类的数据读取器。
2. 通过paddle.io.DataLoader创建异步数据读取的迭代器。

首先，创建定义一个继承 paddle.io.Dataset 的类 MnistDataset，代码如下：

In [ ]:
# 创建一个类MnistDataset，继承paddle.io.Dataset 这个类
# MnistDataset的作用和上面load_data()函数的作用相同，均是构建一个迭代器
class MnistDataset(paddle.io.Dataset):
    def __init__(self, mode):
        datafile = './work/datasets/mnist.json.gz'
        data = json.load(gzip.open(datafile))
        # 读取到的数据区分训练集，验证集，测试集
        train_set, val_set, test_set = data
        if mode == 'train':
            # 获得训练数据集
            imgs, labels = train_set[0], train_set[1]
        elif mode == 'val':
            # 获得验证数据集
            imgs, labels = val_set[0], val_set[1]
        elif mode == 'test':
            # 获得测试数据集
            imgs, labels = test_set[0], test_set[1]
        else:
            raise Exception("mode can only be one of ['train', 'valid', 'eval']")
        # 校验数据
        assert len(imgs) == len(labels), \
            "length of train_imgs({}) should be the same as train_labels({})".format(len(imgs), len(labels))
        self.imgs = imgs
        self.labels = labels

    def __getitem__(self, idx):
        img = np.array(self.imgs[idx]).astype('float32')
        label = np.array(self.labels[idx]).astype('int64')
        return img, label

    def __len__(self):
        return len(self.imgs)

在定义完paddle.io.Dataset后，使用[paddle.io.DataLoader](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc1/api/paddle/io/DataLoader_cn.html) API即可实现异步数据读取，数据会由Python线程预先读取，并异步送入一个队列中。

> *class* paddle.io.DataLoader(dataset, batch_size=100, shuffle=True, num_workers=2)

DataLoader支持单进程和多进程的数据加载方式。当 num_workers=0时，使用单进程方式异步加载数据；当 num_workers=n(n>0)时，主进程将会开启n个子进程异步加载数据。
DataLoader返回一个迭代器，迭代的返回dataset中的数据内容；

dataset是支持 map-style 的数据集(可通过下标索引样本)， map-style 的数据集请参考 [paddle.io.Dataset](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc1/api/paddle/io/Dataset_cn.html) API。

使用paddle.io.DataLoader API以batch的方式进行迭代数据，代码如下：

In [ ]:
# 声明数据加载函数，使用训练模式，MnistDataset构建的迭代器每次迭代只返回batch=1的数据
train_dataset = MnistDataset(mode='train')
# 使用paddle.io.DataLoader 定义DataLoader对象用于加载Python生成器产生的数据，
# DataLoader 返回的是一个批次数据迭代器，并且是异步的；
data_loader = paddle.io.DataLoader(train_dataset, batch_size=100, shuffle=True)
# 迭代的读取数据并打印数据的形状
for i, data in enumerate(data_loader()):
    images, labels = data
    print(i, images.shape, labels.shape)
    if i>=2:
        break

0 [100, 784] [100]
1 [100, 784] [100]
2 [100, 784] [100]


## 基于前馈神经网络实现手写数字识别任务

前馈神经网络作为最基础的神经网络，一般包含输入层、隐藏层和输出层。每层神经元只和相邻层神经元相连，即每层神经元只接收相邻前序神经层中神经元所传来的信息，只给相邻后续神经层中神经元传递信息。在前馈神经网络中，同一层的神经元之间没有任何连接，后续神经层不向其前序相邻神经层传递任何信息。前馈神经网络是目前应用最为广泛的神经网络之一。

### 损失函数
由于手写数字识别任务是一个分类任务，因此我们这里主要介绍交叉熵损失函数。

交叉熵损失函数的设计是基于最大似然思想：最大概率得到观察结果的假设是真的。如何理解呢？举个例子来说，如 **图9** 所示。有两个外形相同的盒子，甲盒中有99个白球，1个蓝球；乙盒中有99个蓝球，1个白球。一次试验取出了一个蓝球，请问这个球应该是从哪个盒子中取出的？

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/13a942e5ec7f4e91badb2f4613c6f71a00e51c8afb6a435e94a0b47cedac9515" width="400" hegiht="" ></center>
<center><br>图9：体会最大似然的思想 </br></center>
<br></br>


相信大家简单思考后均会得出更可能是从乙盒中取出的，因为从乙盒中取出一个蓝球的概率更高$（P(D|h)）$，所以观察到一个蓝球更可能是从乙盒中取出的$(P(h|D))$。$D$是观测的数据，即蓝球白球；$h$是模型，即甲盒乙盒。这就是贝叶斯公式所表达的思想：

$$P(h|D) ∝ P(h) \cdot P(D|h)$$

依据贝叶斯公式，某二分类模型“生成”$n$个训练样本的概率：

$$P(x_1)\cdot S(w^{T}x_1)\cdot P(x_2)\cdot(1-S(w^{T}x_2))\cdot … \cdot P(x_n)\cdot S(w^{T}x_n)$$

------
**说明：**

对于二分类问题，模型为$S(w^{T}x_i)$，$S$为Sigmoid函数。当$y_i$=1，概率为$S(w^{T}x_i)$；当$y_i$=0，概率为$1-S(w^{T}x_i)$。

------

经过公式推导，使得上述概率最大等价于最小化交叉熵，得到交叉熵的损失函数。交叉熵的公式如下：

$$ L = -[\sum_{k=1}^{n} t_k\log y_k +(1- t_k)\log(1-y_k)] $$
  
其中，$\log$表示以$e$为底数的自然对数。$y_k$代表模型输出，$t_k$代表各个标签。$t_k$中只有正确解的标签为1，其余均为0（one-hot表示）。

因此，交叉熵只计算对应着“正确解”标签的输出的自然对数。比如，假设正确标签的索引是“2”，与之对应的神经网络的输出是0.6，则交叉熵误差是$−\log 0.6 = 0.51$；若“2”对应的输出是0.1，则交叉熵误差为$−\log 0.1 = 2.30$。由此可见，交叉熵误差的值是由正确标签所对应的输出结果决定的。


### 模型构建
接下来我们将使用单层的前馈神经网络构建一个手写数字识别模型，并使用该模型的准确率作为 baseline 结果，模型实现代码如下。

In [ ]:
# 定义mnist数据识别网络结构
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()
        # 定义一层全连接层
        self.fc1 = paddle.nn.Linear(in_features=784, out_features=500)
        self.act = paddle.nn.Sigmoid()
        # self.act = paddle.nn.ReLU()
        self.fc2 = paddle.nn.Linear(in_features=500, out_features=10)
        self.softmax = paddle.nn.Softmax()

    # 定义网络结构的前向计算过程
    def forward(self, x):
        outputs = self.fc1(x)
        outputs = self.act(outputs)
        outputs = self.fc2(outputs)
        outputs = self.softmax(outputs)
        return outputs

### 模型训练、评估与保存
这里我们创建一个 Trainer 类，该类用于执行训练过程，计算模型分类效果以及保存模型。
#### 训练
train()用于执行整个训练过程；train_epoch()用于执行一个epoch的训练过程；train_step()用于执行一个批次的训练过程；

#### 评估
准确率是一个直观衡量分类模型效果的指标，为了能够充分的反应不同方法之间的优略，这里我们实现了 val_epoch() 函数来计算模型的分类准确率。

#### 保存
save()用于保存模型参数，这里调用了paddle的 save() API；

In [ ]:
import paddle.nn.functional as F

class Trainer(object):
    def __init__(self, model_path, model, optimizer):
        self.model_path = model_path   # 模型存放路径 
        self.model = model             # 定义的模型
        self.optimizer = optimizer     # 优化器

    def save(self):
        # 保存模型
        paddle.save(self.model.state_dict(), self.model_path)

    def val_epoch(self, datasets):
        self.model.eval()  # 将模型设置为评估状态
        acc = list()
        for batch_id, data in enumerate(datasets()):
            images, labels = data
            pred = self.model(images)   # 获取预测值
            # 取 pred 中得分最高的索引作为分类结果
            pred = paddle.argmax(pred, axis=-1)  
            res = paddle.equal(pred, labels)
            res = paddle.cast(res, dtype='float32')
            acc.extend(res.numpy())  # 追加
        acc = np.array(acc).mean()
        return acc

    def train_step(self, data):
        images, labels = data
        # 前向计算的过程
        predicts = self.model(images)
        # 计算损失
        loss = F.cross_entropy(predicts, labels)
        avg_loss = paddle.mean(loss)
        # 后向传播，更新参数的过程
        avg_loss.backward()
        self.optimizer.step()
        self.optimizer.clear_grad()
        return avg_loss

    def train_epoch(self, datasets, epoch):
        self.model.train()
        for batch_id, data in enumerate(datasets()):
            loss = self.train_step(data)
            # 每训练了1000批次的数据，打印下当前Loss的情况
            if batch_id % 500 == 0:
                print("epoch_id: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))

    def train(self, train_datasets, val_datasets, epochs):
        for i in range(epochs):
            self.train_epoch(train_datasets, i)
            train_acc = self.val_epoch(train_datasets)
            val_acc = self.val_epoch(val_datasets)
            print("epoch_id: {}, train acc is: {}, val acc is {}".format(i, train_acc, val_acc))
        self.save()

In [ ]:
epochs = 10
lr = 0.1
model_path = './mnist.pdparams'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.312789]
epoch_id: 0, batch_id: 500, loss is: [2.1883173]
epoch_id: 0, batch_id: 1000, loss is: [1.9044656]
epoch_id: 0, batch_id: 1500, loss is: [1.8275037]
epoch_id: 0, train acc is: 0.6457599997520447, val acc is 0.6516000032424927
epoch_id: 1, batch_id: 0, loss is: [1.8717408]
epoch_id: 1, batch_id: 500, loss is: [1.8278725]
epoch_id: 1, batch_id: 1000, loss is: [1.751034]
epoch_id: 1, batch_id: 1500, loss is: [1.5611895]
epoch_id: 1, train acc is: 0.8088399767875671, val acc is 0.8220999836921692
epoch_id: 2, batch_id: 0, loss is: [1.7536268]
epoch_id: 2, batch_id: 500, loss is: [1.6741691]
epoch_id: 2, batch_id: 1000, loss is: [1.6057963]
epoch_id: 2, batch_id: 1500, loss is: [1.6794984]
epoch_id: 2, train acc is: 0.8274000287055969, val acc is 0.836899995803833
epoch_id: 3, batch_id: 0, loss is: [1.685547]
epoch_id: 3, batch_id: 500, loss is: [1.6138568]
epoch_id: 3, batch_id: 1000, loss is: [1.6314843]
epoch_id: 3, batch_id: 1500, loss is: [

上述模型中的训练配置如下：
* epoch:10
* lr:0.1
* batch_size:32
* optimizer:SGD

训练 10 轮后模型在验证集的上的准确率为 85.03%。

## 模型优化
在前面的实验中我们使用单层的前馈神经网络取得了一个 baseline 的结果，模型的分类准确率为 85.03%。接下来我们通过调整学习率、激活函数、设置更多的隐藏层以及加入正则化的方式来给大家演示如何一步步的提升模型的准确率。

### 学习率
在深度学习神经网络模型中，通常使用标准的随机梯度下降算法更新参数，学习率代表参数更新幅度的大小，即步长。当学习率最优时，模型的有效容量最大，最终能达到的效果最好。学习率和深度学习任务类型有关，合适的学习率往往需要大量的实验和调参经验。探索学习率最优值时需要注意如下两点：

- **学习率不是越小越好**。学习率越小，损失函数的变化速度越慢，意味着我们需要花费更长的时间进行收敛，如 **图10** 左图所示。
- **学习率不是越大越好**。只根据总样本集中的一个批次计算梯度，抽样误差会导致计算出的梯度不是全局最优的方向，且存在波动。在接近最优解时，过大的学习率会导致参数在最优解附近震荡，损失难以收敛，如 **图2** 右图所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/1e0f066dc9fa4e2bbc942447bdc0578c2ffc6afc15684154ae84bcf31b298d7b" width="500" hegiht="" ></center>
<center><br>图10: 不同学习率（步长过大/过小）的示意图</br></center>
<br></br>

在训练前，我们往往不清楚一个特定问题设置成怎样的学习率是合理的，因此在训练时可以尝试调小或调大，通过观察 Loss下降的情况或者模型在训练集和验证集上的准确率判断合理的学习率。

通过分析单层前馈神经网络+sigmoid的训练过程，发现模型在训练集和验证集上的效果都比较差，初步分析可能是由于训练过程中学习率过小导致的，现在我们通过增大学习率来看看模型的效果是否有所提升。

In [ ]:
epochs = 10
lr = 1.0
model_path = './mnist.pdparams'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.2946477]
epoch_id: 0, batch_id: 500, loss is: [1.7119272]
epoch_id: 0, batch_id: 1000, loss is: [1.6096951]
epoch_id: 0, batch_id: 1500, loss is: [1.6192926]
epoch_id: 0, train acc is: 0.8270000219345093, val acc is 0.838699996471405
epoch_id: 1, batch_id: 0, loss is: [1.564203]
epoch_id: 1, batch_id: 500, loss is: [1.5500519]
epoch_id: 1, batch_id: 1000, loss is: [1.6423697]
epoch_id: 1, batch_id: 1500, loss is: [1.6797929]
epoch_id: 1, train acc is: 0.8360999822616577, val acc is 0.8446000218391418
epoch_id: 2, batch_id: 0, loss is: [1.5447772]
epoch_id: 2, batch_id: 500, loss is: [1.6620564]
epoch_id: 2, batch_id: 1000, loss is: [1.5597005]
epoch_id: 2, batch_id: 1500, loss is: [1.7301197]
epoch_id: 2, train acc is: 0.8452399969100952, val acc is 0.8500000238418579
epoch_id: 3, batch_id: 0, loss is: [1.6207557]
epoch_id: 3, batch_id: 500, loss is: [1.5245409]
epoch_id: 3, batch_id: 1000, loss is: [1.5826328]
epoch_id: 3, batch_id: 1500, loss is:

通过将学习率从 0.1 增大至 1.0 之后，发现模型在验证集上的准确率由之前的 85.03% 提升至 95.87%，由此可见我们之前猜测“学习率过小”的想法是正确的。

### 激活函数
这里将 relu 替换为 sigmoid 之后的模型结构代码如下。

In [ ]:
# 定义mnist数据识别网络结构
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()
        # 定义一层全连接层
        self.fc1 = paddle.nn.Linear(in_features=784, out_features=500)
        self.act = paddle.nn.ReLU()
        self.fc2 = paddle.nn.Linear(in_features=500, out_features=10)
        self.softmax = paddle.nn.Softmax()

    # 定义网络结构的前向计算过程
    def forward(self, x):
        outputs = self.fc1(x)
        outputs = self.act(outputs)
        outputs = self.fc2(outputs)
        outputs = self.softmax(outputs)
        return outputs

In [ ]:
epochs = 10
lr = 0.1
model_path = './mnist.pdparams'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.3012307]
epoch_id: 0, batch_id: 500, loss is: [1.7447678]
epoch_id: 0, batch_id: 1000, loss is: [1.6842225]
epoch_id: 0, batch_id: 1500, loss is: [1.5943805]
epoch_id: 0, train acc is: 0.8307399749755859, val acc is 0.8314999938011169
epoch_id: 1, batch_id: 0, loss is: [1.776561]
epoch_id: 1, batch_id: 500, loss is: [1.6330607]
epoch_id: 1, batch_id: 1000, loss is: [1.5804596]
epoch_id: 1, batch_id: 1500, loss is: [1.6410818]
epoch_id: 1, train acc is: 0.844980001449585, val acc is 0.8445000052452087
epoch_id: 2, batch_id: 0, loss is: [1.5953135]
epoch_id: 2, batch_id: 500, loss is: [1.6094263]
epoch_id: 2, batch_id: 1000, loss is: [1.5781956]
epoch_id: 2, batch_id: 1500, loss is: [1.6458168]
epoch_id: 2, train acc is: 0.8498600125312805, val acc is 0.8479999899864197
epoch_id: 3, batch_id: 0, loss is: [1.5822909]
epoch_id: 3, batch_id: 500, loss is: [1.6323045]
epoch_id: 3, batch_id: 1000, loss is: [1.6275837]
epoch_id: 3, batch_id: 1500, loss is:

通过将激活函数替换为 relu，模型在验证集上的准确率由之前的 85.03% 提升至 96.18%，由此可见 relu 在一定程度上要优于 sigmoid 激活函数。

### 多层前馈神经网络
单层前馈神经网络只包括输入层、隐藏层以及输出层，与单层前馈神经网络相比多层前馈神经网络有多个隐藏层，这意味着多层前馈神经网络有更强的表达能力。这里我们以两个隐藏层的前馈神经网络为例，代码如下。

In [ ]:
# 定义mnist数据识别网络结构，同房价预测网络
class MultiMNIST(paddle.nn.Layer):
    def __init__(self):
        super(MultiMNIST, self).__init__()
        # 定义一层全连接层，输出维度是1
        self.fc1 = paddle.nn.Linear(in_features=784, out_features=512)
        self.act = paddle.nn.ReLU()
        self.fc2 = paddle.nn.Linear(in_features=512, out_features=256)
        self.fc3 = paddle.nn.Linear(in_features=256, out_features=10)
        self.softmax = paddle.nn.Softmax()

    # 定义网络结构的前向计算过程
    def forward(self, x):
        outputs = self.fc1(x)
        outputs = self.act(outputs)
        outputs = self.fc2(outputs)
        outputs = self.act(outputs)
        outputs = self.fc3(outputs)
        outputs = self.softmax(outputs)
        return outputs

In [ ]:
epochs = 10
lr = 0.1
model_path = './mnist.pdparams'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MultiMNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.296736]
epoch_id: 0, batch_id: 500, loss is: [1.7456415]
epoch_id: 0, batch_id: 1000, loss is: [1.6584079]
epoch_id: 0, batch_id: 1500, loss is: [1.5470879]
epoch_id: 0, train acc is: 0.9096400141716003, val acc is 0.916700005531311
epoch_id: 1, batch_id: 0, loss is: [1.618499]
epoch_id: 1, batch_id: 500, loss is: [1.5540549]
epoch_id: 1, batch_id: 1000, loss is: [1.4781168]
epoch_id: 1, batch_id: 1500, loss is: [1.5316998]
epoch_id: 1, train acc is: 0.9342399835586548, val acc is 0.9348999857902527
epoch_id: 2, batch_id: 0, loss is: [1.547365]
epoch_id: 2, batch_id: 500, loss is: [1.4734488]
epoch_id: 2, batch_id: 1000, loss is: [1.521854]
epoch_id: 2, batch_id: 1500, loss is: [1.5706499]
epoch_id: 2, train acc is: 0.9439799785614014, val acc is 0.9435999989509583
epoch_id: 3, batch_id: 0, loss is: [1.5178621]
epoch_id: 3, batch_id: 500, loss is: [1.4757226]
epoch_id: 3, batch_id: 1000, loss is: [1.4958832]
epoch_id: 3, batch_id: 1500, loss is: [1

通过在单层前馈神经网路中添加一层隐藏层，模型在验证集上的准确率由之前的 96.18% 提升至 97.10%，模型的精度得到了进一步的提升。

### 正则化
#### 过拟合现象

对于样本量有限、但需要使用强大模型的复杂任务，模型很容易出现过拟合的表现，即在训练集上的损失小，在验证集或测试集上的损失较大，如 **图11** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/99b879c21113494a9d7315eeda74bc4c8fea07f984824a03bf8411e946c75f1b" width="400" hegiht="" ></center>
<center><br>图11：过拟合现象，训练误差不断降低，但测试误差先降后增</br></center>
<br></br>

反之，如果模型在训练集和测试集上均损失较大，则称为欠拟合。过拟合表示模型过于敏感，学习到了训练数据中的一些误差，而这些误差并不是真实的泛化规律（可推广到测试集上的规律）。欠拟合表示模型还不够强大，还没有很好的拟合已知的训练样本，更别提测试样本了。因为欠拟合情况容易观察和解决，只要训练loss不够好，就不断使用更强大的模型即可，因此实际中我们更需要处理好过拟合的问题。

#### 导致过拟合原因

造成过拟合的原因是模型过于敏感，而训练数据量太少或其中的噪音太多。

如**图12** 所示，理想的回归模型是一条坡度较缓的抛物线，欠拟合的模型只拟合出一条直线，显然没有捕捉到真实的规律，但过拟合的模型拟合出存在很多拐点的抛物线，显然是过于敏感，也没有正确表达真实规律。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/53c389bb3c824706bd2fbc05f83ab0c6dd6b5b2fdedb4150a17e16a1b64c243e" width="700" hegiht="" ></center>
<center><br>图12：回归模型的过拟合，理想和欠拟合状态的表现</br></center>
<br></br>

如**图13** 所示，理想的分类模型是一条半圆形的曲线，欠拟合用直线作为分类边界，显然没有捕捉到真实的边界，但过拟合的模型拟合出很扭曲的分类边界，虽然对所有的训练数据正确分类，但对一些较为个例的样本所做出的妥协，高概率不是真实的规律。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b5a46f7e0fbe4f8686a71d9a2d330ed09f23bca565a44e0d941148729fd2f7d7" width="700" hegiht="" ></center>
<center><br>图13：分类模型的欠拟合，理想和过拟合状态的表现</br></center>
<br></br>

#### 过拟合的成因与防控

为了更好的理解过拟合的成因，可以参考侦探定位罪犯的案例逻辑，如 **图14** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/34de60a675b64468a2c3fee0844a168d53e891eaacf643fd8c1c9ba8e3812bcc" width="700" hegiht="" ></center>
<center><br>图14：侦探定位罪犯与模型假设示意</br></center>
<br></br>

**对于这个案例，假设侦探也会犯错，通过分析发现可能的原因：**

1. 情况1：罪犯证据存在错误，依据错误的证据寻找罪犯肯定是缘木求鱼。

2. 情况2：搜索范围太大的同时证据太少，导致符合条件的候选（嫌疑人）太多，无法准确定位罪犯。

**那么侦探解决这个问题的方法有两种：或者缩小搜索范围（比如假设该案件只能是熟人作案），或者寻找更多的证据。**

**归结到深度学习中，假设模型也会犯错，通过分析发现可能的原因：**

1. 情况1：训练数据存在噪音，导致模型学到了噪音，而不是真实规律。

2. 情况2：使用强大模型（表示空间大）的同时训练数据太少，导致在训练数据上表现良好的候选假设太多，锁定了一个“虚假正确”的假设。

**对于情况1，我们使用数据清洗和修正来解决。 对于情况2，我们或者限制模型表示能力，或者收集更多的训练数据。**

而清洗训练数据中的错误，或收集更多的训练数据往往是一句“正确的废话”，在任何时候我们都想获得更多更高质量的数据。在实际项目中，更快、更低成本可控制过拟合的方法，只有限制模型的表示能力。

#### 正则化项

为了防止模型过拟合，在没有扩充样本量的可能下，只能降低模型的复杂度，可以通过限制参数的数量或可能取值（参数值尽量小）实现。

具体来说，在模型的优化目标（损失）中人为加入对参数规模的惩罚项。当参数越多或取值越大时，该惩罚项就越大。通过调整惩罚项的权重系数，可以使模型在“尽量减少训练损失”和“保持模型的泛化能力”之间取得平衡。泛化能力表示模型在没有见过的样本上依然有效。正则化项的存在，增加了模型在训练集上的损失。

飞桨支持为所有参数加上统一的正则化项，也支持为特定的参数添加正则化项。前者的实现如下代码所示，仅在优化器中设置``weight_decay``参数即可实现。使用参数``coeff``调节正则化项的权重，权重越大时，对模型复杂度的惩罚越高。

In [ ]:
epochs = 10
lr = 0.1
model_path = './mnist.pdparams'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MultiMNIST()
opt = paddle.optimizer.SGD(learning_rate=lr,
                           weight_decay=paddle.regularizer.L2Decay(coeff=5e-4),
                           parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.3049517]
epoch_id: 0, batch_id: 500, loss is: [1.6930485]
epoch_id: 0, batch_id: 1000, loss is: [1.5100982]
epoch_id: 0, batch_id: 1500, loss is: [1.5393964]
epoch_id: 0, train acc is: 0.843779981136322, val acc is 0.8483999967575073
epoch_id: 1, batch_id: 0, loss is: [1.5944157]
epoch_id: 1, batch_id: 500, loss is: [1.5461731]
epoch_id: 1, batch_id: 1000, loss is: [1.5863192]
epoch_id: 1, batch_id: 1500, loss is: [1.5110033]
epoch_id: 1, train acc is: 0.9290199875831604, val acc is 0.9304999709129333
epoch_id: 2, batch_id: 0, loss is: [1.5418378]
epoch_id: 2, batch_id: 500, loss is: [1.5766435]
epoch_id: 2, batch_id: 1000, loss is: [1.5307435]
epoch_id: 2, batch_id: 1500, loss is: [1.5195917]
epoch_id: 2, train acc is: 0.9450399875640869, val acc is 0.9463000297546387
epoch_id: 3, batch_id: 0, loss is: [1.4880548]
epoch_id: 3, batch_id: 500, loss is: [1.5822808]
epoch_id: 3, batch_id: 1000, loss is: [1.515212]
epoch_id: 3, batch_id: 1500, loss is:

通过在多层前馈神经网络的训练过程中添加正则化项，模型在验证集上的准确率由之前的 97.10% 提升至 97.18%，模型的精度得到了略微提升。同时模型在训练集和验证集上准确率的差距得到了进一步的缩小，从而证明正则化的有效性。

## 可视化分析
训练模型时，经常需要观察模型的评价指标，分析模型的优化过程，以确保训练是有效的。飞桨提供了专业的作图工具 VisualDL，VisualDL是飞桨可视化分析工具，以丰富的图表呈现训练参数变化趋势、模型结构、数据样本、高维数据分布等。帮助用户清晰直观地理解深度学习模型训练过程及模型结构，进而实现高效的模型调优，具体代码实现如下。

* 步骤1：引入VisualDL库，定义作图数据存储位置（供第3步使用），本案例的路径是“log”。
```
from visualdl import LogWriter

def get_summary_writer(self):
	if self.summary_dir is None:
		return None
	else:
		return LogWriter(self.summary_dir)
        
def update_summary(self, **kwargs): 
	if self.summary_writer is None:
		pass
	else:
		for name in kwargs:
			self.summary_writer.add_scalar(tag=name, step=self.global_step, value=kwargs[name])
```
* 步骤2：在训练过程中插入作图语句。当每个batch训练完成后，将当前损失作为一个新增的数据点存储到第一步设置的文件中。使用变量 global_step 记录下已经训练的批次数，作为作图的X轴坐标。

```python
def train_epoch(self, datasets, epoch):
	self.model.train()
	for batch_id, data in enumerate(datasets()):
		loss = self.train_step(data)
		self.update_summary(train_loss=loss.numpy())
		self.global_step += 1
```

完整代码如下。

In [ ]:
# 安装VisualDL
!pip install --upgrade --pre visualdl

In [ ]:
from visualdl import LogWriter
import paddle.nn.functional as F


class Trainer(object):
    def __init__(self, model_path, model, optimizer, summary_dir=None):
        self.model_path = model_path
        self.summary_dir = summary_dir   # 作图文件存储目录
        self.model = model
        self.optimizer = optimizer
        self.summary_writer = self.get_summary_writer()  # 获取作图对象
        self.global_step = 0

    def get_summary_writer(self):
        """实例化作图对象"""
        if self.summary_dir is None:
            return None
        else:
            return LogWriter(self.summary_dir)

    def update_summary(self, **kwargs):  
        """向作图对象中插入语句"""
        if self.summary_writer is None:
            pass
        else:
            for name in kwargs:
                self.summary_writer.add_scalar(tag=name, step=self.global_step, value=kwargs[name])

    def save(self):
        paddle.save(self.model.state_dict(), self.model_path)

    def val_epoch(self, datasets):
        self.model.eval()
        acc = list()
        for batch_id, data in enumerate(datasets()):
            images, labels = data
            pred = self.model(images)
            pred = paddle.argmax(pred, axis=-1)  # 取 pred 中得分最高的索引作为分类结果
            res = paddle.equal(pred, labels)
            res = paddle.cast(res, dtype='float32')
            acc.extend(res.numpy())  # 追加
        acc = np.array(acc).mean()
        return acc

    def train_step(self, data):
        images, labels = data

        # 前向计算的过程
        predicts = self.model(images)

        # 计算损失
        loss = F.cross_entropy(predicts, labels)
        avg_loss = paddle.mean(loss)

        # 后向传播，更新参数的过程
        avg_loss.backward()
        self.optimizer.step()
        self.optimizer.clear_grad()
        return avg_loss

    def train_epoch(self, datasets, epoch):
        self.model.train()
        for batch_id, data in enumerate(datasets()):
            loss = self.train_step(data)
            # 记录训练损失
            self.update_summary(train_loss=loss.numpy())
            self.global_step += 1

            # 每训练了1000批次的数据，打印下当前Loss的情况
            if batch_id % 500 == 0:
                print("epoch_id: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))

    def train(self, train_datasets, val_datasets, epochs):
        for i in range(epochs):
            self.train_epoch(train_datasets, i)
            train_acc = self.val_epoch(train_datasets)
            val_acc = self.val_epoch(val_datasets)
            self.update_summary(train_acc=train_acc, val_acc=val_acc)
            print("epoch_id: {}, train acc is: {}, val acc is {}".format(i, train_acc, val_acc))
        self.save()

In [ ]:
epochs = 10
lr = 1.0
model_path = './mnist.pdparams'
summary_dir = './work/summary'

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt,
    summary_dir=summary_dir
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.3144398]
epoch_id: 0, batch_id: 500, loss is: [1.6106814]
epoch_id: 0, batch_id: 1000, loss is: [1.6772048]
epoch_id: 0, batch_id: 1500, loss is: [1.5089396]
epoch_id: 0, train acc is: 0.8843200206756592, val acc is 0.8916000127792358
epoch_id: 1, batch_id: 0, loss is: [1.6163015]
epoch_id: 1, batch_id: 500, loss is: [1.5770197]
epoch_id: 1, batch_id: 1000, loss is: [1.6005037]
epoch_id: 1, batch_id: 1500, loss is: [1.4678018]
epoch_id: 1, train acc is: 0.9097999930381775, val acc is 0.9133999943733215
epoch_id: 2, batch_id: 0, loss is: [1.478539]
epoch_id: 2, batch_id: 500, loss is: [1.5579653]
epoch_id: 2, batch_id: 1000, loss is: [1.5488136]
epoch_id: 2, batch_id: 1500, loss is: [1.464844]
epoch_id: 2, train acc is: 0.9307199716567993, val acc is 0.9320999979972839
epoch_id: 3, batch_id: 0, loss is: [1.4766693]
epoch_id: 3, batch_id: 500, loss is: [1.5465415]
epoch_id: 3, batch_id: 1000, loss is: [1.5130544]
epoch_id: 3, batch_id: 1500, loss is:

* 步骤3：命令行启动VisualDL。

使用“visualdl --logdir [数据文件所在文件夹路径] 的命令启动VisualDL。在VisualDL启动后，命令行会打印出可用浏览器查阅图形结果的网址。
``` 
$ visualdl --logdir ./log --port 8080
```

* 步骤4：打开浏览器，查看作图结果，如下图所示。

查阅的网址在第三步的启动命令后会打印出来（如http://127.0.0.1:8080/），将该网址输入浏览器地址栏刷新页面的效果如下图所示。除了右侧对数据点的作图外，左侧还有一个控制板，可以调整诸多作图的细节。


<center><img src="https://github.com/lovejing0306/Images/blob/master/Case/mnist/VisualDL.jpg?raw=true" width="800" hegiht="" ></center>
<center><br>VisualDL作图示例</br></center>
<br></br>


## 资源配置
使用飞桨构建的模型可以指定运行的资源，通过 paddle.set_device API，设置在 GPU 上训练还是 CPU 上训练。

> paddle.set_device *(device)*

参数 
device (str)：此参数确定特定的运行设备，可以是`cpu`、 `gpu:x`或者是`xpu:x`。其中，`x`是GPU或XPU的编号。当device是`cpu`时， 程序在CPU上运行；当device是`gpu:x`时，程序在GPU上运行。

修改后的代码如下：

In [12]:
epochs = 10
lr = 1.0
use_gpu = False
model_path = './mnist.pdparams'

paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

train_dataset = MnistDataset(mode='train')
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=32,
                                    shuffle=True,
                                    num_workers=4)

val_dataset = MnistDataset(mode='val')
val_loader = paddle.io.DataLoader(val_dataset, batch_size=128)

model = MNIST()
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [2.2991498]
epoch_id: 0, batch_id: 500, loss is: [1.772263]
epoch_id: 0, batch_id: 1000, loss is: [1.6144236]
epoch_id: 0, batch_id: 1500, loss is: [1.5906253]
epoch_id: 0, train acc is: 0.8229399919509888, val acc is 0.8270999789237976
epoch_id: 1, batch_id: 0, loss is: [1.6858568]
epoch_id: 1, batch_id: 500, loss is: [1.578338]
epoch_id: 1, batch_id: 1000, loss is: [1.5622997]
epoch_id: 1, batch_id: 1500, loss is: [1.6056569]
epoch_id: 1, train acc is: 0.8350200057029724, val acc is 0.8353999853134155
epoch_id: 2, batch_id: 0, loss is: [1.7619798]
epoch_id: 2, batch_id: 500, loss is: [1.6406751]
epoch_id: 2, batch_id: 1000, loss is: [1.5183274]
epoch_id: 2, batch_id: 1500, loss is: [1.5920503]
epoch_id: 2, train acc is: 0.8440799713134766, val acc is 0.8424999713897705
epoch_id: 3, batch_id: 0, loss is: [1.5738984]
epoch_id: 3, batch_id: 500, loss is: [1.6155759]
epoch_id: 3, batch_id: 1000, loss is: [1.6095839]
epoch_id: 3, batch_id: 1500, loss is: